# Comparing different Neural Networks

Each Network uses a vector of the eigen-eigen parts, the rest-rest parts and the eigenvalues and computes the rest-eigen contribution

In [ ]:
using LmaPredict, Flux, Statistics

## Reading the data

In [ ]:
const path_config = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Daten Simon/dat"
const path_plot = "../plots"

In [ ]:
fname = readdir(path_config)[2:5001]
idx = sortperm( parse.(Int64, fname))
fname = fname[idx]
em_n = "VV"

cnfgarr = Vector{LMAConfig}(undef, 0)
for f in fname
    push!(cnfgarr, get_LMAConfig(joinpath(path_config, f), "g5-g5", em=em_n, bc=false))
end

## Splitting data in training and test sets

In [ ]:
# Select a specific Tsource and divide data into training and test set for eigenvalues, rr re and ee components
TSRC = "24"
NCNFG = length(cnfgarr)
TVALS = length(cnfgarr[1].data["rr"][TSRC]) - 1
if em_n == "PA"
    EIGVALS = 32
else 
    EIGVALS = 64
end

eigvals_data_train = Array{Float32}(undef, EIGVALS, 500)
rr_data_train = Array{Float32}(undef, TVALS, 500)
ee_data_train = Array{Float32}(undef, TVALS, 500)
re_data_train = Array{Float32}(undef, TVALS, 500)

eigvals_data_test = Array{Float64}(undef, EIGVALS, 4500)
rr_data_test = Array{Float64}(undef, TVALS, 4500)
ee_data_test = Array{Float64}(undef, TVALS, 4500)
re_data_test = Array{Float64}(undef, TVALS, 4500)

for (k, dd) in enumerate(getfield.(cnfgarr, :data)[1:500])
    eigvals_data_train[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_train[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    ee_data_train[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_train[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
end
for (k, dd) in enumerate(getfield.(cnfgarr, :data)[501:5000])
    eigvals_data_test[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_test[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    ee_data_test[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_test[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
end

## Defining training and test data

In [ ]:
input_length = 2*TVALS + EIGVALS
output_length = TVALS

input_shape_train = vcat(1 ./ eigvals_data_train, ee_data_train, rr_data_train)
output_shape_train = re_data_train

input_shape_test = vcat(1 ./ eigvals_data_test, ee_data_test, rr_data_test)
output_shape_test = re_data_test;

### Input data, normalized and standardized

In [ ]:
max_input_train = maximum(input_shape_train)
min_input_train = minimum(input_shape_train)

mean_input_train = mean(mean.([input_shape_train[:,i] for i in 1:train_size]))
std_input_train = std(mean.([input_shape_train[:,i] for i in 1:train_size]))

input_data_train_normalized = (input_shape_train .- max_input_train) ./ (max_input_train - min_input_train)
input_data_train_standardized = (input_shape_train .- mean_input_train) ./ std_input_train

input_data_test_normalized = (input_shape_test .- max_input_train) ./ (max_input_train - min_input_train)
input_data_test_standardized = (input_shape_test .- mean_input_train) ./ std_input_train;

### Output data, normalized and standardized

In [ ]:
max_output_train = maximum(output_shape_train)
min_output_train = minimum(output_shape_train)

mean_output_train = mean(mean.([output_shape_train[:,i] for i in 1:train_size]))
std_output_train = std(mean.([output_shape_train[:,i] for i in 1:train_size]))

output_data_train_normalized = (output_shape_train .- max_output_train) ./ (max_output_train - min_output_train)
output_data_train_standardized = (output_shape_train .- mean_output_train) ./ std_output_train;

## Describing different Neural Networks

We want to compare different Networks with respect to the parameters:

- How many trainable variables has the Network?
- How long does training take?
- How good is the perfomance - measured by the standard deviation of the difference - with respect to
     - How many configurations have been used for the bias correction

In [ ]:
activation_function = NNlib.tanh

models = [
    Chain(
    Dense(input_length => 1, activation_function),
    Dense(1 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 10, activation_function),
    Dense(10 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 10, activation_function),
    Dense(10 => 10, activation_function),
    Dense(10 => output_length, activation_function),
    ),
];

In [ ]:
function loss(flux_model, x, y)
    ŷ = flux_model(x)
    Flux.mse(ŷ, y, agg=sum)
end

In [ ]:
using Flux:params

optimizer = Flux.Adam(0.000001)
loss_discription = "MSE"

epochs = 100_000
batch_size = 32

percentages_bc = [0.0, 0.01, 0.02, 0.05, 0.1, 0.12]
n_configs_bc = Int.(4500 .* percentages_bc)

loader = Flux.DataLoader((input_data_train, target_train), batchsize=batch_size, shuffle=true)

for model in models
    parameters = 0
    layers = params(model)
    for layer in layers
        parameters += length(hcat(layer...))
    end

    outputDirectory = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Tool Allesandro/repo/LmaPredict/benchmarks/ee+rr+ev/$parameters"

    optim = Flux.setup(optimizer, model)
    
    function training()
        losses = []
        for epoch in 1:epochs
            for (x, y) in loader
                grads = gradient(m -> loss(m, x, y), model)
                Flux.update!(optim, model, grads[1])
                push!(losses,loss(model, x, y))
            end
        end
        return losses
    end

    losses = training()
    
    out_of_sample_predictions = (model(input_data_test_normalized) .* (max_output_train - min_output_train)) .+ max_output_train

    analyse_predictions(
        out_of_sample_predictions,
        output_shape_test,
        TSRC,
        EIGVALS,
        model,
        optimizer,
        loss_function,
        loss_discription,
        epochs,
        batch_size,
        losses,
        outputDirectory
    )
end